# Import libraries

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip3 install pydicom
!pip3 install segmentation_models
!pip3 install backbone-network

     |████████████████████████████████| 1.9MB 5.6MB/s 
     |████████████████████████████████| 51kB 4.6MB/s 


In [ ]:
import warnings
warnings.filterwarnings('ignore')

import os
import cv2
import csv
import pickle
import pydicom
import numpy as np
import pandas as pd 
from glob import glob
import keras
import tensorflow as tf
from keras import backend as K

os.chdir('/content/drive/MyDrive/Initial-Code')

from iou_dice_loss_functions import my_iou_metric, iou_metric_batch_val, bce_dice_loss
%env SM_FRAMEWORK=tf.keras

from model_rle_functions import predict_result_val
from data_generator_functions import data_generator, label_generator

import seg_models
keras.backend.set_image_data_format('channels_last')

from keras.optimizers import SGD
from keras.callbacks import ModelCheckpoint

import sys
sys.path.insert(0, 'siim-acr-pneumothorax-segmentation')

seed = 1994
np.random.seed = seed
os.environ['PYTHONHASHSEED'] = str(seed)
tf.seed = seed

# Dataset

In [ ]:
# defining configuration parameters
org_size = 1024 # original image size
img_size = 256 # image resize size
batch_size = 40 # batch size for training unet

## Load train and validation data from files

In [ ]:
pkl_file_train = open('process_data/X_train.pkl', 'rb')

X_train = pickle.load(pkl_file_train)
print(X_train)

['../Dataset/stage1_sim/1.2.276.0.7230010.3.1.4.8323329.1702.1517875169.44421.dcm'
 '../Dataset/stage1_sim/1.2.276.0.7230010.3.1.4.8323329.5196.1517875186.906815.dcm'
 '../Dataset/stage1_sim/1.2.276.0.7230010.3.1.4.8323329.2310.1517875172.88826.dcm'
 ...
 '../Dataset/stage1_sim/1.2.276.0.7230010.3.1.4.8323329.6776.1517875201.61444.dcm'
 '../Dataset/stage1_sim/1.2.276.0.7230010.3.1.4.8323329.2039.1517875170.835799.dcm'
 '../Dataset/stage1_sim/1.2.276.0.7230010.3.1.4.8323329.10187.1517875222.531834.dcm']


In [ ]:
pkl_file_val = open('process_data/X_val.pkl', 'rb')

X_val = pickle.load(pkl_file_val)

In [ ]:
pkl_file_masks = open('process_data/masks.pkl', 'rb')

masks = pickle.load(pkl_file_masks)

## Data generation & Augmentations

In [ ]:
import albumentations as A

In [ ]:
training_augmentation = A.Compose([
    A.HorizontalFlip(p=0.5),
    A.OneOf([
        A.RandomContrast(),
        A.RandomGamma(),
        A.RandomBrightness(),
         ], p=0.3),
    A.OneOf([
        A.ElasticTransform(alpha=120, sigma=120 * 0.05, alpha_affine=120 * 0.03),
        A.GridDistortion(),
        A.OpticalDistortion(distort_limit=2, shift_limit=0.5),
        ], p=0.3),
    A.ShiftScaleRotate(shift_limit=0.2, scale_limit=0.2, rotate_limit=20,
                                        interpolation=cv2.INTER_LINEAR, border_mode=cv2.BORDER_CONSTANT, p=1),
    A.RandomSizedCrop(min_max_height=(206,256), height=img_size, width=img_size,p=0.25)
],p=1)

In [ ]:
params_train = {'img_size': img_size,
          'batch_size': batch_size,
          
          'shuffle': True,
           'augmentations':training_augmentation,
           }

params_val = {'img_size': img_size,
          'batch_size': batch_size,
          'n_channels': 1,
          'shuffle': True,
         }

training_generator = data_generator(X_train, masks, **params_train)
validation_generator = data_generator(X_val, masks, **params_val)

# Segmentation model

In [ ]:
K.clear_session()

In [ ]:
BACKBONE = 'resnet34'
model = seg_models.Unet(backbone_name=BACKBONE, encoder_weights='imagenet')
model.summary()

In [ ]:
opt = SGD(momentum=0.9)

In [ ]:
model.compile(optimizer=opt, loss=bce_dice_loss, metrics=[my_iou_metric])

In [ ]:
from swa import SWA
from annealing_scheduler_function import CosineAnnealingScheduler

In [ ]:
epochs = 50
swa = SWA('model_output/512_resnet34_swa_exp1_stage1.model',45)

callbacks = [
    ModelCheckpoint("model_output/512_resnet34_exp1_stage1.model",monitor='val_loss', 
                            mode = 'min', save_best_only=True,
                            verbose=1),
    swa,
    CosineAnnealingScheduler(T_max=epochs, eta_max=1e-3, eta_min=1e-5, verbose=1)
]

In [ ]:
history = model.fit_generator(generator=training_generator,
                            validation_data=validation_generator,   
                           epochs=epochs, verbose=1,
                            callbacks=callbacks)

In [ ]:
# list all data in history
import matplotlib.pyplot as plt
 
print(history.history.keys())

# summarize history for iou
plt.figure(figsize=(20,5))
plt.subplot(1,2,1)
plt.plot(history.history['my_iou_metric'])
plt.plot(history.history['val_my_iou_metric'])
plt.title('model IOU')
plt.ylabel('iou')
plt.xlabel('epoch')
plt.legend(['train', 'Validation'], loc='upper left')

# summarize history for loss
plt.subplot(1,2,2)
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'Validation'], loc='upper left')

In [ ]:
# Load best model or swa model

model.load_weights('model_output/512_resnet34_swa_exp1_stage1.model')

In [ ]:
# defining configuration parameters
org_size = 1024 # original image size
img_size = 512# image resize size
batch_size = 10 # batch size for training unet

In [ ]:
pkl_file_train = open('process_data/X_train.pkl', 'rb')

X_train = pickle.load(pkl_file_train)

In [ ]:
pkl_file_val = open('process_data/X_val.pkl', 'rb')

X_val = pickle.load(pkl_file_val)

In [ ]:
pkl_file_masks = open('process_data/masks.pkl', 'rb')

masks = pickle.load(pkl_file_masks)

In [ ]:
import albumentations as A

In [ ]:
training_augmentation = A.Compose([
    A.HorizontalFlip(p=0.5),
    A.OneOf([
        A.RandomContrast(),
        A.RandomGamma(),
        A.RandomBrightness(),
         ], p=0.3),
    A.OneOf([
        A.ElasticTransform(alpha=120, sigma=120 * 0.05, alpha_affine=120 * 0.03),
        A.GridDistortion(),
        A.OpticalDistortion(distort_limit=2, shift_limit=0.5),
        ], p=0.3),
    A.ShiftScaleRotate(shift_limit=0.2, scale_limit=0.2, rotate_limit=20,
                                        interpolation=cv2.INTER_LINEAR, border_mode=cv2.BORDER_CONSTANT, p=1),
    A.RandomSizedCrop(min_max_height=(412, 512), height=img_size, width=img_size,p=0.25)
],p=1)

In [ ]:
params_train = {'img_size': img_size,
          'batch_size': batch_size,
          'n_channels': 3,
          'shuffle': True,
           'augmentations':training_augmentation,
           }

params_val = {'img_size': img_size,
          'batch_size': batch_size,
          'n_channels': 3,
          'shuffle': True,
         }

# Generators
training_generator = data_generator(X_train, masks, **params_train)
validation_generator = data_generator(X_val, masks, **params_val)

In [ ]:
from swa import SWA
from annealing_scheduler_function import CosineAnnealingScheduler

In [ ]:
epochs = 60
swa = SWA('model_output/512_resnet34_swa_exp1_stage2.model',55)

callbacks = [
    ModelCheckpoint("model_output/512_resnet34_exp1_stage2.model",monitor='val_loss', 
                            mode = 'min', save_best_only=True,
                            verbose=1),
    swa,
    CosineAnnealingScheduler(T_max=epochs, eta_max=1e-3, eta_min=1e-5, verbose=1)
]

In [ ]:
history = model.fit_generator(generator=training_generator,
                            validation_data=validation_generator,   
                           epochs=epochs, verbose=1,
                            callbacks=callbacks) 

In [ ]:
# list all data in history
import matplotlib.pyplot as plt
 
print(history.history.keys())

# summarize history for iou
plt.figure(figsize=(20,5))
plt.subplot(1,2,1)
plt.plot(history.history['my_iou_metric'])
plt.plot(history.history['val_my_iou_metric'])
plt.title('model IOU')
plt.ylabel('iou')
plt.xlabel('epoch')
plt.legend(['train', 'Validation'], loc='upper left')

# summarize history for loss
plt.subplot(1,2,2)
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'Validation'], loc='upper left')

# Evaluation validation data

In [ ]:
# Load best model or swa model

model.load_weights('model_output/512_resnet34_swa_exp1_stage1.model')

In [ ]:
params_val = {'img_size': img_size,
          'batch_size': 5,
          'n_channels': 3,
          'shuffle': False,
         }

# Generators
validation_generator = data_generator(X_val, masks, **params_val)

In [ ]:
AUGMENTATIONS_TEST_FLIPPED = A.Compose([
    A.HorizontalFlip(p=1),
],p=1)

params_val_flip = {'img_size': img_size,
          'batch_size': 5,
          'n_channels': 3,
          'shuffle': False,
        'augmentations':AUGMENTATIONS_TEST_FLIPPED,
         }

validation_generator_flipped = data_generator(X_val, masks, **params_val_flip)

In [ ]:
preds_valid_orig = predict_result_val(model,validation_generator,img_size)
preds_valid_flipped = predict_result_val(model,validation_generator_flipped,img_size)
preds_valid_flipped = np.array([np.fliplr(x) for x in preds_valid_flipped])
preds_valid = 0.5*preds_valid_orig + 0.5*preds_valid_flipped

In [ ]:
np.savez_compressed('process_data/val_pre/preds_valid_resnet34', array1= preds_valid)

In [ ]:
y_truth_val = label_generator(X_val, masks, len(preds_valid), img_size, 3)

np.savez_compressed('process_data/val_pre/y_truth_val', array1= y_truth_val)

In [ ]:
decompressed_array= np.load("process_data/val_pre/y_truth_val.npz")  
y_truth_val = decompressed_array['array1']

In [ ]:
## Scoring for last model
score = 0.0
mask_area = 0
best_th = 0

thresholds = np.arange(0.2, 0.9, 0.01) 
areas = [1024, 2048, 3072, 4096]
for threshold in tqdm(thresholds):
    for area in tqdm(areas):
        iou = iou_metric_batch_val(y_truth_val, np.int32(preds_valid > threshold), area)
        if iou > score:
            score = iou
            mask_area = area
            best_th = threshold
            print("Threshold {}\tMask area {}\tIoU {}".format(best_th, mask_area, score))
    print()

# Evaluation validation data stage 2


In [ ]:
# Load best model or swa model

model.load_weights('model_output/512_resnet34_swa_exp1_stage2.model')

In [ ]:
params_val = {'img_size': img_size,
          'batch_size': 5,
          'n_channels': 3,
          'shuffle': False,
         }

# Generators
validation_generator = data_generator(X_val, masks, **params_val)

In [ ]:
AUGMENTATIONS_TEST_FLIPPED = A.Compose([
    A.HorizontalFlip(p=1),
],p=1)

params_val_flip = {'img_size': img_size,
          'batch_size': 5,
          'n_channels': 3,
          'shuffle': False,
        'augmentations':AUGMENTATIONS_TEST_FLIPPED,
         }

validation_generator_flipped = data_generator(X_val, masks, **params_val_flip)

In [ ]:
preds_valid_orig = predict_result_val(model,validation_generator,img_size)
preds_valid_flipped = predict_result_val(model,validation_generator_flipped,img_size)
preds_valid_flipped = np.array([np.fliplr(x) for x in preds_valid_flipped])
preds_valid = 0.5*preds_valid_orig + 0.5*preds_valid_flipped

In [ ]:
np.savez_compressed('process_data/val_pre/preds_valid_resnet34', array1= preds_valid)

In [ ]:
y_truth_val = label_generator(X_val, masks, len(preds_valid), img_size, 3)

np.savez_compressed('process_data/val_pre/y_truth_val', array1= y_truth_val)

In [ ]:
decompressed_array= np.load("process_data/val_pre/y_truth_val.npz")  
y_truth_val = decompressed_array['array1']

In [ ]:
## Scoring for last model
score = 0.0
mask_area = 0
best_th = 0

thresholds = np.arange(0.2, 0.9, 0.01) 
areas = [1024, 2048, 3072, 4096]
for threshold in tqdm(thresholds):
    for area in tqdm(areas):
        iou = iou_metric_batch_val(y_truth_val, np.int32(preds_valid > threshold), area)
        if iou > score:
            score = iou
            mask_area = area
            best_th = threshold
            print("Threshold {}\tMask area {}\tIoU {}".format(best_th, mask_area, score))
    print()